# Introduction

# Methods

## Data

The dataset used in this case study is a set of Monte Carlo simulations of signals for process that produce Higgs bosons and background processes that do not produce Higgs bosons\footnotemark\footnotetext{https://archive.ics.uci.edu/ml/datasets/HIGGS}.
The dataset contains 11 million instances of 28 features: 21 kenimatic properties measured by particle deterors in the accelerator and 7 engineered features.
The target variable is a binary indicator where 1 indicates a Higgs processes and 0 indicates a background process.
The reference paper indicates that the last 500,000 instances in this training set were used for model validation.
We maintained this train-validation split in this case study, using the last 500,000 instances for validation and the prior instances for training.

## Neural Network

### Replication of Model

In this case study, we replicated the modeling performed by Baldi, Sadowski, & Whiteson on the Higgs boson dataset with deep neural networks (DNN).
The model used in the reference study was a 5-layer multi-perceptron (MLP) with `tanh` activation, weight decay ($L2$ reularization) coefficient of $1 x 10^{5}$, and layers initialized with weights from the random normal. 
These hyperparamters are summerized in table 1.

**Table 1. Model Hyperparameters**



In addition to the model architecture, we also replicated the training process.
The model was trainined with stochastic gradient descent (SGD) with a batch size of 100.
The learning rate was initialized at 0.05 and decreased by a factor of 1.0000002 on each batch to a minumum rate of $1 x 10^{-6}$.
The momentum was initialized to 0.9 and increated linearly to 0.99 over 200 epochs, remaining constant after the 200th epoch.
For the stopping criterion, the reference paper indicates that early stopping with minimum change in error of 0.00001 over 10 epochs was used to determine when to stop the training process (resulting in training the model over 200-1000 epochs).
However, the reference paper does not indicate what error metric was monitored for early stopping.
We moitored the validation binary cross-entropy loss for early stopping as is typical practice in deep learning.
The training process is summerized in table 2.
In this study, the model was implemented with `TensorFlow`\footnotemark\footnotetext{https://pypi.org/project/tensorflow/2.2.0/} (version 2.2.0) because the framework used in the reference paper, `PyLearn2`\footnotemark\footnotetext{http://deeplearning.net/software/pylearn2/}, is no long actively maintained.


**Table 2. Model Training Parameters**

### Suggestions for Model Improvement

Include in your report:
Based on the class notes and discussion suggest improvements to the procedure. What are standard practices now versus when this paper was written? What kind of improvements do they provide?

* dropout
* ResNets (skip connections)
* optimizers: Adam, RMSprop
* layer initialization

# Results

How would you quantify if your result duplicated the paper’s? **-> one sample t-test**

# Conclusions

# References

* \[1\] Baldi, P., P. Sadowski, and D. Whiteson. “Searching for Exotic Particles in High-energy Physics with Deep Learning.” Nature Communications 5 (July 2, 2014). [https://arxiv.org/pdf/1402.4735.pdf](https://arxiv.org/pdf/1402.4735.pdf)

# Code

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.regularizers import l2
print(tf.__version__)

auc_score = tf.keras.metrics.AUC()
seed = 42

2.2.0


In [2]:
data = pd.read_csv('./data/HIGGS.csv', header = None)
# from the paper: The last 500,000 examples are used as a test set.
# The first column is the class label (1 for signal, 0 for background), followed by the 28 features
train_test_split = data.shape[0] - 500000
X_test = data.iloc[ train_test_split : , 1: ]
y_test = data.iloc[ train_test_split : , 0 ]
X_train = data.iloc[ : train_test_split , 1: ]
y_train = data.iloc[ : train_test_split , 0 ]

In [3]:
# helper functions

def create_model(hidden_size,
                 first_layer_init,
                 hidden_layer_init,
                 output_layer_init,
                 weight_decay,
                 starting_lr,
                 metrics,
                 ):
    """Create the model used in this case study
    """
    model = Sequential([
        layers.Dense(hidden_size, activation='tanh',
                    kernel_initializer=first_layer_init,
                    kernel_regularizer=l2(weight_decay)
                    ),
        layers.Dense(hidden_size, activation='tanh',
                    kernel_initializer=hidden_layer_init,
                    kernel_regularizer=l2(weight_decay)
                    ),
        layers.Dense(hidden_size, activation='tanh',
                    kernel_initializer=hidden_layer_init,
                    kernel_regularizer=l2(weight_decay)
                    ),
        layers.Dense(hidden_size, activation='tanh',
                    kernel_initializer=hidden_layer_init,
                    kernel_regularizer=l2(weight_decay)
                    ),
        layers.Dense(hidden_size, activation='tanh',
                    kernel_initializer=hidden_layer_init,
                    kernel_regularizer=l2(weight_decay)
                    ),
        layers.Dense(1, activation='sigmoid',
                    kernel_initializer=output_layer_init,
                    kernel_regularizer=l2(weight_decay)
                    )
    ])

    model.compile(optimizer=optimizers.SGD(lr=starting_lr),
                  loss='binary_crossentropy',
                  metrics=metrics)
    
    return model

def plot_training_curves(history, title=None, save_path=None):
    ''' Plot the training curves for loss and accuracy given a model history
    '''
    # find the minimum loss epoch
    minimum = np.min(history.history['val_loss'])
    min_loc = np.where(minimum == history.history['val_loss'])[0]
    # get the vline y-min and y-max
    loss_min, loss_max = (min(history.history['val_loss'] + history.history['loss']),
                          max(history.history['val_loss'] + history.history['loss']))
    acc_min, acc_max = (min(history.history['val_accuracy'] + history.history['accuracy']),
                        max(history.history['val_accuracy'] + history.history['accuracy']))
    # create figure
    fig, ax = plt.subplots(ncols=2, figsize = (15,7))
    fig.suptitle(title)
    index = np.arange(1, len(history.history['accuracy']) + 1)
    # plot the loss and validation loss
    ax[0].plot(index, history.history['loss'], label = 'loss')
    ax[0].plot(index, history.history['val_loss'], label = 'val_loss')
    ax[0].vlines(min_loc + 1, loss_min, loss_max, label = 'min_loss_location')
    ax[0].set_title('Loss')
    ax[0].set_ylabel('Loss')
    ax[0].set_xlabel('Epochs')
    ax[0].legend()
    # plot the accuracy and validation accuracy
    ax[1].plot(index, history.history['accuracy'], label = 'accuracy')
    ax[1].plot(index, history.history['val_accuracy'], label = 'val_accuracy')
    ax[1].vlines(min_loc + 1, acc_min, acc_max, label = 'min_loss_location')
    ax[1].set_title('Accuracy')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_xlabel('Epochs')
    ax[1].legend()
    plt.show()
    if save_path is not None:
        fig.savefig(save_path)

In [4]:
#We selected a five-layer neural
#network with 300 hidden units in each layer, a learning
#rate of 0.05, and a weight decay coefficient of 1 × 10−5

hidden_size = 300
starting_lr = 0.05
weight_decay = 1e-6

#Hidden units all used the tanh activation function.
#Weights were initialized from a normal distribution with
#zero mean and standard deviation 0.1 in the first layer,
#0.001 in the output layer, and 0.05 all other hidden layers. 
#Gradient computations were made on mini-batches
#of size 100. 

first_layer_init = initializers.RandomNormal(
    mean=0.0, stddev=0.1, seed=seed
)
hidden_layer_init = initializers.RandomNormal(
    mean=0.0, stddev=0.05, seed=seed
)
output_layer_init = initializers.RandomNormal(
    mean=0.0, stddev=0.001, seed=seed
)

In [5]:
model = create_model(hidden_size,
                     first_layer_init,
                     hidden_layer_init,
                     output_layer_init,
                     weight_decay,
                     starting_lr,
                     metrics=['accuracy',
                           auc_score])

In [6]:
# The learning rate decayed by a factor
#of 1.0000002 every batch update until it reached a minimum of 10−6
class LRSchedule(callbacks.Callback):
    def on_batch_end(self, batch, logs):
        current_lr = K.get_value(model.optimizer.lr)
        if current_lr > 1e-6:
            lr = current_lr / 1.0000002 #1.00002
            K.set_value(self.model.optimizer.lr, lr)
        else:
            K.set_value(self.model.optimizer.lr, 1e-6)

lr_scheduler = LRSchedule()

In [7]:
#A momentum term increased linearly over
#the first 200 epochs from 0.9 to 0.99, at which point it
#remained constant. 

class MomentumSchedule(callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        starting_value = 0.9
        ending_value = 0.99
        number_epochs = 200
        step_increase = (ending_value-starting_value) / number_epochs
        if epoch > number_epochs:
            K.set_value(self.model.optimizer.momentum, ending_value)
        else:
            current_momentum = K.get_value(self.model.optimizer.momentum)
            current_momentum += step_increase
            K.set_value(self.model.optimizer.momentum, step_increase)

momentum_scheduler = MomentumSchedule()

In [8]:
#Training ended when the momentum had
#reached its maximum value and the minimum error on
#the validation set (500,000 examples) had not decreased
#by more than a factor of 0.00001 over 10 epochs

early_stopping_criterion = callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.00001,
    patience=10
)

In [ ]:
hist = model.fit(X_train, y_train,
                 batch_size=100,
                 epochs=400,
                 validation_data=(X_test, y_test),
                 callbacks=[
                     lr_scheduler, momentum_scheduler, early_stopping_criterion
                 ]
                )

Epoch 1/400

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

105000/105000 [==============================] - 545s 5ms/step - loss: 0.5449 - accuracy: 0.7178 - auc: 0.7950 - val_loss: 0.5050 - val_accuracy: 0.7474 - val_auc: 0.8311
Epoch 2/400
 44271/105000 [===========>..................] - ETA: 5:01 - loss: 0.5009 - accuracy: 0.7505 - auc: 0.8329

In [ ]:
plot_training_curves(hist)